In [45]:
from searchPlus import *

In [46]:
line1 = "## ## ## ## ## ## ## ## ##\n"
line2 = "## .. .. .. .. .. .. .. ##\n"
line3 = "## .. .. .. 00 .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. .. ##\n"
line7 = "## .. .. .. 01 .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "## ## ## ## ## ## ## ## ##\n"
grid = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

In [47]:
# class PenguinsPairs(Problem):
    
#     def __init__(self, ice_map=grid):
#         self.initial = self.parse_grid(ice_map)
#         self.rows = len(self.initial)
#         self.cols = len(self.initial[0])
#         super().__init__(self.initial)
    
#     def parse_grid(self, ice_map):
#         return tuple(tuple(line.split()) for line in ice_map.strip().split("\n"))
    
#     def actions(self, state):
#         moves = []
#         directions = [('E', (0, 1)), ('N', (-1, 0)), ('O', (0, -1)), ('S', (1, 0))]
        
#         for penguin in ('00', '01'):
#             for i in range(self.rows):
#                 for j in range(self.cols):
#                     if state[i][j] == penguin:
#                         for d, (di, dj) in directions:
#                             ni, nj = i + di, j + dj
#                             if 0 <= ni < self.rows and 0 <= nj < self.cols and state[ni][nj] == "..":
#                                 moves.append((penguin, d))
#         return moves
    
#     def result(self, state, action):
#         penguin, direction = action
#         directions = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'O': (0, -1)}
#         di, dj = directions[direction]
        
#         new_state = [list(row) for row in state]
#         for i in range(self.rows):
#             for j in range(self.cols):
#                 if new_state[i][j] == penguin:
#                     ni, nj = i + di, j + dj
#                     if 0 <= ni < self.rows and 0 <= nj < self.cols and new_state[ni][nj] == "..":
#                         new_state[i][j] = ".."
#                         new_state[ni][nj] = penguin
#                         return tuple(tuple(row) for row in new_state)
        
#         return state
    
#     def goal_test(self, state):
#         # return any(state[i][j] == "00" and state[i+1][j] == "01" for i in range(self.rows - 1) for j in range(self.cols))
#         for i in range(self.rows):
#             for j in range(self.cols - 1):
#                 if state[i][j] == "00" and state[i][j+1] == "01":
#                     return True
            
#             for j in range(self.cols):
#                 if i < self.rows - 1 and state[i][j] == "00" and state[i+1][j] == "01":
#                     return True
#         return False
    
#     def display(self, state):
#         return "\n".join(" ".join(row) for row in state) + "\n"
    
#     def executa(self, state, actions_list, verbose=False):
#         cost = 0
#         for a in actions_list:
#             seg = self.result(state, a)
#             cost = self.path_cost(cost, state, a, seg)
#             state = seg
#             obj = self.goal_test(state)
#             if verbose:
#                 print('Ação:', a)
#                 print(self.display(state), end='')
#                 print('Custo Total:', cost)
#                 print('Atingido o objectivo?', obj)
#                 print()
#             if obj:
#                 break
#         return (state, cost, obj)

In [48]:
class PenguinsPairs(Problem):
    def __init__(self, ice_map=grid):
        self.initial = self.parse_grid(ice_map)
        self.rows = len(self.initial)
        self.cols = len(self.initial[0])
        super().__init__(self.initial)

    def parse_grid(self, ice_map):
        return tuple(tuple(line.split()) for line in ice_map.strip().split("\n"))

    def actions(self, state):
        moves = []
        directions = [('E', (0, 1)), ('N', (-1, 0)), ('O', (0, -1)), ('S', (1, 0))]
        
        for penguin in sorted([p for row in state for p in row if p.isdigit()]):
            for i in range(self.rows):
                for j in range(self.cols):
                    if state[i][j] == penguin:
                        for d, (di, dj) in directions:
                            ni, nj = i + di, j + dj
                            while 0 <= ni < self.rows and 0 <= nj < self.cols and state[ni][nj] == "..":
                                ni += di
                                nj += dj
                            if 0 <= ni < self.rows and 0 <= nj < self.cols and state[ni][nj] not in ('()', penguin):
                                moves.append((penguin, d))
        
        return sorted(moves)

    def result(self, state, action):
        penguin, direction = action
        directions = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'O': (0, -1)}
        di, dj = directions[direction]
        
        new_state = [list(row) for row in state]
        
        for i in range(self.rows):
            for j in range(self.cols):
                if new_state[i][j] == penguin:
                    ni, nj = i + di, j + dj
                    while 0 <= ni < self.rows and 0 <= nj < self.cols and new_state[ni][nj] == "..":
                        ni += di
                        nj += dj
                    ni -= di
                    nj -= dj
                    
                    new_state[i][j] = ".."
                    new_state[ni][nj] = penguin
                    return tuple(tuple(row) for row in new_state)
        
        return state
    
    def goal_test(self, state):
        return any(
            state[i][j] == "00" and state[i+1][j] == "01"
            for i in range(self.rows - 1) for j in range(self.cols)
        )

    def display(self, state):
        return "\n".join(" ".join(row) for row in state) + "\n"
    
    def executa(self, state, actions_list, verbose=False):
        """Executa uma sequência de acções a partir do estado devolvendo o triplo formado pelo estado final, 
        pelo custo acumulado e pelo booleano que indica se o objectivo foi ou não atingido. Se o objectivo 
        for atingido antes da sequência ser atingida, devolve-se o estado e o custo corrente.
        Há o modo verboso e o não verboso, por defeito."""
        cost = 0
        for a in actions_list:
            seg = self.result(state, a)
            cost = self.path_cost(cost, state, a, seg)
            state = seg
            obj = self.goal_test(state)
            if verbose:
                print('Ação:', a)
                print(self.display(state), end='')
                print('Custo Total:', cost)
                print('Atingido o objectivo?', obj)
                print()
            if obj:
                break
        return (state, cost, obj)

In [49]:
p = PenguinsPairs(grid)
print(p.actions(p.initial))

[('00', 'E'), ('00', 'N'), ('00', 'O'), ('01', 'E'), ('01', 'O'), ('01', 'S')]


In [50]:
print(p.display(p.initial))

## ## ## ## ## ## ## ## ##
## .. .. .. .. .. .. .. ##
## .. .. .. 00 .. .. .. ##
## .. .. .. .. .. .. .. ##
## .. .. () () () .. .. ##
## .. .. .. .. .. .. .. ##
## .. .. .. 01 .. .. .. ##
## .. .. .. .. .. .. .. ##
## ## ## ## ## ## ## ## ##



In [51]:
line1 = "## () () () () () () () ##\n"
line2 = "## .. .. .. .. .. .. .. ##\n"
line3 = "## .. .. .. 00 .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. .. ##\n"
line7 = "## .. .. .. 01 .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "## () () () () () () () ##\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

In [52]:
p2 = PenguinsPairs(grid2)
print(p2.actions(p2.initial))

[('00', 'E'), ('00', 'O'), ('01', 'E'), ('01', 'O')]


In [53]:
p = PenguinsPairs()
seq = [('00', 'E'),('01', 'E'),('00', 'S')]
p.goal_test(p.executa(p.initial, seq)[0])

True